In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from colorcloud.biasutti2019riu import SemanticSegmentationTask, RIUNet
from colorcloud.behley2019iccv import SemanticSegmentationLDM, ProjectionVizTransform
from colorcloud.chen2020mvlidarnet import MVLidarNet
from torch.nn import CrossEntropyLoss
import lightning as L
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime
from torch import nn

In [ ]:
remapping_rules = {
    1: 1,
    4: 2,
    6: 3,
    7: 4,
    2: 4,
    9: 5,
    11: 6
}
data = SemanticSegmentationLDM(train_batch_size=10, remapping_rules=None)
data.setup('fit')
epoch_steps = len(data.train_dataloader())

In [ ]:
n_epochs = 5
learner = SemanticSegmentationTask(
    RIUNet(),
    CrossEntropyLoss(reduction='none'),
    data.viz_tfm, 
    total_steps=n_epochs*epoch_steps
)

In [ ]:
time = datetime.now()
timestamp = str(time.year) + '-' + str(time.month) + '-' + str(time.day) + '_' + str(time.hour) + '-' + str(time.minute) + '-' + str(time.second)

wandb_logger = WandbLogger(project="colorcloud", name=timestamp, log_model="all")
wandb_logger.watch(learner.model, log="all")

In [ ]:
# train model
trainer = L.Trainer(max_epochs=n_epochs, logger=wandb_logger)
trainer.fit(learner, data)

In [ ]:
wandb.finish()